In [1]:
def get_answers_counts(data_subset):
    for idx, col in enumerate(data_subset.columns):
        print(questions[col]['text'])
        print('Count:' + '\t' + 'Answer:')
        val_cnts = data_subset[col].value_counts()
        for pos, ind in enumerate(val_cnts.index):
            count = val_cnts.values[pos]
            if col not in numeric_col:
                print(str(count) + '\t' + questions[col][ind] )
            else:
                print(str(count) + '\t' + str(ind))

In [ ]:
def barplot(data, idx, col, title, numeric_cols, questions):
    plt.figure(idx)
    if col not in numeric_cols:
        val_cnts = data[col].value_counts()
        text = [questions[col][val] for val in val_cnts.index]
        # replace x and y with each other for vertical plot
        ax = sns.barplot(y=text, x=val_cnts.values)
        ax.set_title(title)
        for ind, val in enumerate(val_cnts):
            ax.text(val, ind, val, color='black', ha="center")
        #for plot vertical plot
        #ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    else:
        ax = sns.histplot(x=data[col])
        ax.set_title(title)
        ax.set_xlabel('Age')

In [ ]:
def perform_mapping(model_data, questions_map):
    for index, row in model_data.iterrows():
        for col in model_data.columns:
            if np.isnan(row[col]) or row[col]==0:
                continue
            row[col] = questions_map[col][int(row[col])]
    return model_data